In [5]:
import numpy as np
import sympy as sp 
import numpy.linalg as la
from IPython.display import display, Math, Latex
from fractions import Fraction

In [104]:
fractor = np.vectorize(Fraction)
defractor = np.vectorize(float)

def str_to_np(x_raw):
    x_rows = list(filter(None, x_raw.split('\n')))
    x = np.array(list(map(lambda x: list(map(int, x.split())), x_rows)))
    return x

def texmaker(x):
    x = x.astype(str) # если значения числовые, при приведении к обыкновенным дробям может получиться некрасиво
    x = fractor(x)
    tex = '\\begin{bmatrix}\n  '
    for row in x:
        for ind, el in enumerate(row):
            if el.denominator != 1:
                n = str(el.numerator)
                d = str(el.denominator)
                tex += r'\frac{' + n + '}{' + d + '}'
                if ind < len(row)-1:
                    tex += ' &  '
            else:
                n = str(el.numerator)
                tex += n
                if ind < len(row)-1:
                    tex += ' &  '
        tex += '\\\\\n  '
    tex += '\n\\end{bmatrix}'
    return tex 

def texfrac(x):
    x = Fraction(str(x))
    if x.denominator != 1:
        n = str(x.numerator)
        d = str(x.denominator)
        tex = r'\frac{' + n + '}{' + d + '}'
    else:
        n = str(x.numerator)
        tex = n
    return tex

def texpoly(x):
    pow = len(x) - 1
    tex = '\chi(\lambda) = '
    for i in range(len(x)):
        if x[i] != 0:    #если коэффициент 0 то не пишем слагаемое
            pref = ''
            postf = ''
            if i == 0:    # начинаем со знака
                if x[i] < 0:
                    pref += '-'
            else:
                if x[i] > 0:
                    pref += ' +'
                else:
                    pref += ' -'
            if x[i] != 1:
                pref += str(abs(x[i]))
            pow = len(x) - i - 1
            if pow > 1:
                tex += pref + '\lambda^{' + str(pow) + '}'
            elif pow == 1:
                tex += pref + '\lambda'
            else:
                tex += pref
                if x[i] == 1:
                    tex += ' 1'
            
#         if x[i] == 0:
#             pass
#         elif x[i] == 1:
#             tex += '\lambda^' + str(pow - i)
#         elif x[i] == -1:
#             tex += '-\lambda^' + str(pow - i)
#         elif x[i] > 0 and :
#             tex += '-\lambda^' + str(pow - i)
#         else:
#             tex += str(x[i]) + '\lambda^' + str(pow - i)
#         if pow - i == 1:
#             tex = tex[:-2]    # УЖАСНО!!!! отрезаем первые степени
#         elif pow - i == 0:
#             tex = tex[:-9]    # ЕЩЕ УЖАСНЕЕ!!!! отрезаем нулевые степени
    tex = tex.replace('=  +', '=  ')
    return tex

def calculate_c(a, b):
    display(Math(r'\Large \mathfrak{C} = [B, AB, A^2B, ... A^{n-1}]'))
    c_size = a.shape[0]
    c_goth = b.copy()
    for i in range(1, c_size):
        tmp = np.matmul(la.matrix_power(a, i), b)
        c_goth = np.concatenate((c_goth, tmp), axis=1)
    display(Math(r'\Large \mathfrak{C} = ' + texmaker(c_goth)))
    return c_goth

def kalman_decomposition(a, b, c_goth = None):
    if not c_goth:
        c_goth = calculate_c(a, b)
    c_rank = la.matrix_rank(c_goth)
    c_size = a.shape[0]
    display(Math(r'\Large Rank(\mathfrak{C}) = ' + str(c_rank)))
    display(Latex(r'Подпространство управляемых состояний - $\mathbb{R^' + str(c_rank)+'}$'))
    display(Latex(r'Подпространство неуправляемых состояний - $\mathbb{R^' + str(c_goth.shape[0]-c_rank)+'}$'))
    display(Latex(r'Проверим, является ли система стабилизируемой. Дополняем  до  линейно  независимой  системы:'))
    _, inds = sp.Matrix(c_goth).rref()   # нахождение линейно зависимых векторов
    s = c_goth[:, inds]
    for i in range(c_size - len(inds)):    # каждый проход приклеивается вектор
        for j in range(c_size):    # пробуем единичные векторы 
            s_temp = s.copy()
            test_vec = np.zeros((1, c_size), dtype = int)
            test_vec[0, c_size - 1 - j] += 1 
            s_temp = np.concatenate((s_temp, test_vec.T), axis=1)
            if la.matrix_rank(s_temp) == len(inds) + 1 +i:
                s = np.concatenate((s, test_vec.T), axis=1)
                break
                
    display(Math(r'\Large S = ' + texmaker(s)))
    a_hat = np.matmul(np.matmul(fractor(la.inv(s)), fractor(a)), s)
    display(Math(r'\Large \hat{A} = S^{-1}AS = '
                 + texmaker(la.inv(s)) + texmaker(a)
                 + texmaker(s) + ' = '+ texmaker(a_hat)))
    b_hat = np.matmul(fractor(la.inv(s)), fractor(b))
    display(Math(r'\Large \hat{B} = S^{-1}B = '
                 + texmaker(la.inv(s)) + texmaker(b) 
                 + ' = ' + texmaker(b_hat)))
    display(Math(r'\Large \hat{\dot{x}} = ' + texmaker(a_hat) + '\hat{x} + '
                + texmaker(b_hat) + '\hat{u}'))
    unc_sp = a_hat[c_rank:, c_rank:]
    con_sp = a_hat[:c_rank, :c_rank]
    unc_eig, _ = la.eig(unc_sp.astype(float))
    display(Latex(r'Управляемое подпространство:'))
    display(Math(texmaker(con_sp)))
    display(Latex(r'Неуправляемое подпространство:'))
    display(Math(texmaker(unc_sp)))
    display(Latex(r'Собственные числа неуправляемого подпространства:'))
    unc_eig, _ = la.eig(unc_sp.astype(float))
    ev_str = ''
    for i, ev in enumerate(unc_eig):
        ev_str += '\lambda_' + str(i+1) + ' = ' + texfrac(ev) + ','    # ужасно...
    display(Latex(r'Собственные числа неуправляемого подпространства:'))
    display(Math(ev_str[:-1]))    #КОСТЫЛЬ!!!! который убирает запятую в конце (кринж)
    
    if (unc_eig > 0).any():
        display(Latex(r'Присутствуют неотрицательные собственные числа. Следовательно неуправляемая система не стабилизируема.'))
    else:
        display(Latex(r'Все собственные числа отрицательные. Значит неуправляемая часть устойчива и неуправляемая система стабилизируема.')) 
    
def frobenius_form(a, b, c_goth = None):
    if not c_goth:
        c_goth = calculate_c(a, b)
    c_rank = la.matrix_rank(c_goth)
    display(Math(r'\Large Rank(\mathfrak{C}) = ' + str(c_rank)))
    display(Latex(r'По алгебраическому критерию управляемости Калмана система управляема. Приведем её к канонической форме Фробениуса-Люенбегра:'))
    
    display(Latex(r'Строим характеристический полином матрицы управляемости'))
    M = sp.Matrix(a)
    lamda = sp.symbols('lamda')
    p = M.charpoly(lamda)
    p_coeffs = p.all_coeffs()
    display(Math(texpoly(p_coeffs)))
    if p_coeffs[0] < 0:
        display(Latex(r'Первый коэффициент характеристического полинома отрицательный, домножаем на -1:'))
        p_coeffs = [-cof for cof in p_coeffs]
        display(Math(texpoly(p_coeffs)))
    bot_row = [-p_coeffs[-i] for i in range(1, c_rank + 1)]
    a_wave = np.concatenate((np.delete(np.identity(c_rank), 0, axis=0), [bot_row]), axis=0)
    display(Math(r'\Large \widetilde{A} = ' + texmaker(a_wave)))

In [85]:
kalman_decomposition(a, b)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

In [101]:
frobenius_form(a, b)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [103]:
texpoly([0, 2, 0, 1])

'\\chi(\\lambda) =  +2\\lambda^{2} + 1'

In [107]:
display(Math(texpoly([0, 0, 0, 1])))

<IPython.core.display.Math object>

In [84]:
a_raw = '''
1 -2 0
-2 1 0
2 -2 -1
'''

b_raw = '''
0
1
1
'''
a = str_to_np(a_raw)
b = str_to_np(b_raw)
display(Math(r'\Large \dot{x} = ' + texmaker(a) + 'x + '
            + texmaker(b) + 'u'))

<IPython.core.display.Math object>

In [82]:
a_raw = '''
1 2 3
1 2 2
0 3 4
'''

b_raw = '''
-1
1
0
'''
a = str_to_np(a_raw)
b = str_to_np(b_raw)
display(Math(r'\Large \dot{x} = ' + texmaker(a) + 'x + '
            + texmaker(b) + 'u'))

<IPython.core.display.Math object>

In [84]:
a = str_to_np(a_raw)
b = str_to_np(b_raw)
a, b

(array([[0, 1, 1],
        [1, 4, 5],
        [2, 0, 3]]),
 array([[0],
        [1],
        [0]]))

In [85]:
display(Math(texmaker(b)))

<IPython.core.display.Math object>

In [81]:
display(Math(r'\Large\mathfrak{C}'))

<IPython.core.display.Math object>

In [97]:
la.matrix_power(a, 2) * b.T

array([[ 0,  4,  0],
       [ 0, 17,  0],
       [ 0,  2,  0]])

In [279]:
# c_size = a.shape[0]
# c_goth = b.copy()

# for i in range(1, c_size):
#     tmp = np.matmul(la.matrix_power(a, i), b)
#     c_goth = np.concatenate((c_goth, tmp), axis=1)
    
# c_rank = la.matrix_rank(c_goth)
# display(Math(r'\Large \mathfrak{C} = ' + texmaker(c_goth) + r'\\ \Large Rank(\mathfrak{C}) = ' + str(c_rank)))
# _, inds = sympy.Matrix(c_goth).rref()   # нахождение линейно зависимых векторов
# s = c_goth[:, inds]
# for i in range(c_size - len(inds)):    # каждый проход приклеивается вектор
#     for j in range(c_size):    # пробуем единичные векторы 
#         s_temp = s.copy()
#         test_vec = np.zeros((1, c_size), dtype = int)
#         test_vec[0, c_size - 1 - j] += 1 
#         s_temp = np.concatenate((s_temp, test_vec.T), axis=1)
#         if la.matrix_rank(s_temp) == len(inds) + 1 +i:
#             s = np.concatenate((s, test_vec.T), axis=1)
#             break

# display(Math(r'\Large S = ' + texmaker(s) + r'\\ \Large Rank(S) = ' + str(la.matrix_rank(s))))

In [211]:
a_hat = np.matmul(np.matmul(fractor(la.inv(s)), fractor(a)), s)
display(Math(r'\Large \hat{A} = ' + texmaker(a_hat)))
b_hat = np.matmul(fractor(la.inv(s)), fractor(b))
display(Math(r'\Large \hat{B} = ' + texmaker(b_hat)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [243]:
texfrac(ev)

In [247]:
unc_sp = a_hat[c_rank:, c_rank:]
unc_eig, _ = la.eig(unc_sp.astype(float))
for ev in unc_eig:
    display(Math(texfrac(ev)))
(unc_eig > 0).any()

<IPython.core.display.Math object>

False